In [ ]:
# ========================================================================
# 1. CARREGAMENTO E CONFIGURAÇÃO INICIAL
# ========================================================================

import pandas as pd
from unidecode import unidecode

print("🔄 CARREGANDO ARQUIVO CSV...")

# Carregando o arquivo CSV
df_notas = pd.read_csv('8NotasFW_ProdInfoMesSem.csv', sep=";", encoding="latin1")
df_notas_original = df_notas.copy() #Backup do DataFrame original

print("✅ Arquivo carregado com sucesso!")

# ========================================================================
# 1.1 CONVERSÃO DE TIPOS DE DADOS
# ========================================================================

print("\n🔄 CONVERTENDO TIPOS DE DADOS...")

# Convertendo a coluna Data para datetime
df_notas['Data'] = pd.to_datetime(df_notas['Data'])

print("✅ Conversão de tipos concluída!")

# ========================================================================
# 1.2 INFORMAÇÕES GERAIS DO DATASET
# ========================================================================

print("\n" + "="*60)
print("📊 INFORMAÇÕES GERAIS DO DATASET")
print("="*60)

print(f"📁 Total de registros carregados: {len(df_notas):,}")
print(f"📅 Período dos dados: {df_notas['Data'].min().strftime('%d/%m/%Y')} a {df_notas['Data'].max().strftime('%d/%m/%Y')}")
print(f"📋 Colunas disponíveis: {len(df_notas.columns)}")

print(f"\n📋 Estrutura dos dados:")
for col in df_notas.columns:
    print(f"   • {col}: {df_notas[col].dtype}")

print(f"\n📊 Primeiras 3 linhas:")
print(df_notas.head(3))

🔄 CARREGANDO ARQUIVO CSV...
✅ Arquivo carregado com sucesso!

🔄 CONVERTENDO TIPOS DE DADOS...
✅ Arquivo carregado com sucesso!

🔄 CONVERTENDO TIPOS DE DADOS...
✅ Conversão de tipos concluída!

📊 INFORMAÇÕES GERAIS DO DATASET
📁 Total de registros carregados: 2,536,320
📅 Período dos dados: 02/01/2024 a 31/12/2024
📋 Colunas disponíveis: 13

📋 Estrutura dos dados:
   • Data: datetime64[ns]
   • Controle: int64
   • Cliente: int64
   • Bairro: object
   • ProCod: int64
   • NumSeq: int64
   • Quantidade: float64
   • Preco: float64
   • TotalItem: float64
   • ProNom: object
   • Mes: int64
   • Semana: int64
   • GrpNom: object

📊 Primeiras 3 linhas:
        Data  Controle  Cliente  Bairro  ProCod  NumSeq  Quantidade  Preco  \
0 2024-01-02    630309    99999  CENTRO    1734       1        0.14  35.98   
1 2024-01-02    630315    99999  CENTRO   11502       1        1.00  15.99   
2 2024-01-02    630319    27458  CENTRO    2915       1        0.61   7.39   

   TotalItem                  Pr

In [10]:
# ========================================================================
# 2. DEFINIÇÕES PARA LIMPEZA DE DADOS
# ========================================================================

print("⚙️ CONFIGURANDO PARÂMETROS DE LIMPEZA...")

# ========================================================================
# 2.1 CÓDIGOS A SEREM EXCLUÍDOS (conforme infos.md)
# ========================================================================

# Códigos de serviços
codigos_servicos = [63998, 64003, 63997, 63996, 63994, 63995]

# Códigos de fretes  
codigos_fretes = [1107, 1124, 1135, 1152, 1071, 1111, 1078, 48018]

# Lista completa para exclusão
codigos_excluir = codigos_servicos + codigos_fretes

print(f"📋 Códigos de serviços a excluir: {len(codigos_servicos)}")
print(f"📋 Códigos de fretes a excluir: {len(codigos_fretes)}")
print(f"📋 Total de códigos a excluir: {len(codigos_excluir)}")

# ========================================================================
# 2.2 MAPEAMENTO PARA PADRONIZAÇÃO DE BAIRROS
# ========================================================================

# Dicionário de mapeamento para padronizar nomes dos bairros
mapeamento_bairros = {
    'BAIRRO SANTO INACIO': 'SANTO INACIO',
    'LOT BOA ESPERANCA': 'BOA ESPERANCA',
    'LOTEAMENTO BOA ESPERNCA': 'BOA ESPERANCA',
    'LOTEAMENTO BOA ESPERANCA': 'BOA ESPERANCA',
    'ITAPAJE': 'ITAPAGE',
    'JARDIM PRIMAVERA': 'PRIMAVERA'
}

print(f"📋 Bairros a padronizar: {len(mapeamento_bairros)}")

# ========================================================================
# 2.3 VERIFICAÇÃO INICIAL
# ========================================================================

# Verificando quantos registros têm códigos de serviço/frete
registros_servicos_fretes = df_notas[df_notas['ProCod'].isin(codigos_excluir)]
print(f"\n📊 Registros com códigos de serviços/fretes: {len(registros_servicos_fretes):,}")

# Verificando bairros que serão padronizados
for bairro_antigo, bairro_novo in mapeamento_bairros.items():
    if bairro_antigo in df_notas['Bairro'].values:
        qtd = len(df_notas[df_notas['Bairro'] == bairro_antigo])
        print(f"📊 {bairro_antigo}: {qtd:,} registros → {bairro_novo}")

print("\n✅ Configurações de limpeza definidas!")

⚙️ CONFIGURANDO PARÂMETROS DE LIMPEZA...
📋 Códigos de serviços a excluir: 6
📋 Códigos de fretes a excluir: 8
📋 Total de códigos a excluir: 14
📋 Bairros a padronizar: 6

📊 Registros com códigos de serviços/fretes: 3,570
📊 BAIRRO SANTO INACIO: 14 registros → SANTO INACIO
📊 LOT BOA ESPERANCA: 588 registros → BOA ESPERANCA
📊 BAIRRO SANTO INACIO: 14 registros → SANTO INACIO
📊 LOT BOA ESPERANCA: 588 registros → BOA ESPERANCA
📊 LOTEAMENTO BOA ESPERNCA: 391 registros → BOA ESPERANCA
📊 LOTEAMENTO BOA ESPERANCA: 1,987 registros → BOA ESPERANCA
📊 LOTEAMENTO BOA ESPERNCA: 391 registros → BOA ESPERANCA
📊 LOTEAMENTO BOA ESPERANCA: 1,987 registros → BOA ESPERANCA
📊 ITAPAJE: 1,908 registros → ITAPAGE
📊 JARDIM PRIMAVERA: 20,782 registros → PRIMAVERA

✅ Configurações de limpeza definidas!
📊 ITAPAJE: 1,908 registros → ITAPAGE
📊 JARDIM PRIMAVERA: 20,782 registros → PRIMAVERA

✅ Configurações de limpeza definidas!


In [ ]:
# ========================================================================
# 3. EXCLUSÃO DE SERVIÇOS E FRETES
# ========================================================================

print("🗑️ REMOVENDO SERVIÇOS E FRETES...")

print(f"📊 Registros antes da exclusão: {len(df_notas):,}")

# Filtrando apenas produtos (excluindo serviços e fretes)
df_notas = df_notas[~df_notas['ProCod'].isin(codigos_excluir)].copy()

registros_removidos = len(df_notas) + len(codigos_excluir) - len(df_notas)
print(f"📊 Registros após exclusão: {len(df_notas):,}")
print(f"📊 Registros removidos: {registros_servicos_fretes.shape[0]:,}")

print("✅ Serviços e fretes removidos com sucesso!")

🗑️ REMOVENDO SERVIÇOS E FRETES...
📊 Registros antes da exclusão: 2,536,320
📊 Registros após exclusão: 2,532,750
📊 Registros removidos: 3,570
✅ Serviços e fretes removidos com sucesso!
📊 Registros após exclusão: 2,532,750
📊 Registros removidos: 3,570
✅ Serviços e fretes removidos com sucesso!


In [ ]:
# ========================================================================
# 4. PADRONIZAÇÃO DOS NOMES DOS BAIRROS
# ========================================================================

print("🏘️ PADRONIZANDO NOMES DOS BAIRROS...")

# Contando bairros únicos antes da padronização
bairros_antes = df_notas['Bairro'].nunique()
print(f"📊 Bairros únicos antes: {bairros_antes}")

# Aplicando as substituições
print("\n📝 Substituições realizadas:")
for bairro_antigo, bairro_novo in mapeamento_bairros.items():
    if bairro_antigo in df_notas['Bairro'].values:
        qtd_registros = len(df_notas[df_notas['Bairro'] == bairro_antigo])
        print(f"   {bairro_antigo} → {bairro_novo} ({qtd_registros:,} registros)")
        
        # Fazendo a substituição
        df_notas['Bairro'] = df_notas['Bairro'].replace(bairro_antigo, bairro_novo)

# Contando bairros únicos após padronização
bairros_depois = df_notas['Bairro'].nunique()
print(f"\n📊 Bairros únicos após: {bairros_depois}")
print(f"📊 Reduções realizadas: {bairros_antes - bairros_depois}")

print("\n✅ Padronização de bairros concluída!")

🏘️ PADRONIZANDO NOMES DOS BAIRROS...
📊 Bairros únicos antes: 28

📝 Substituições realizadas:
   BAIRRO SANTO INACIO → SANTO INACIO (13 registros)
   BAIRRO SANTO INACIO → SANTO INACIO (13 registros)
   LOT BOA ESPERANCA → BOA ESPERANCA (588 registros)
   LOT BOA ESPERANCA → BOA ESPERANCA (588 registros)
   LOTEAMENTO BOA ESPERNCA → BOA ESPERANCA (391 registros)
   LOTEAMENTO BOA ESPERNCA → BOA ESPERANCA (391 registros)
   LOTEAMENTO BOA ESPERANCA → BOA ESPERANCA (1,980 registros)
   LOTEAMENTO BOA ESPERANCA → BOA ESPERANCA (1,980 registros)
   ITAPAJE → ITAPAGE (1,908 registros)
   ITAPAJE → ITAPAGE (1,908 registros)
   JARDIM PRIMAVERA → PRIMAVERA (20,756 registros)
   JARDIM PRIMAVERA → PRIMAVERA (20,756 registros)

📊 Bairros únicos após: 22
📊 Reduções realizadas: 6

✅ Padronização de bairros concluída!

📊 Bairros únicos após: 22
📊 Reduções realizadas: 6

✅ Padronização de bairros concluída!


In [ ]:
# ========================================================================
# 5. VERIFICAÇÃO E TRATAMENTO DE DUPLICATAS
# ========================================================================

print("🔍 VERIFICANDO DUPLICATAS...")

# Verificando duplicatas por chave única (Controle + ProCod + NumSeq)
duplicatas_mask = df_notas.duplicated(subset=['Controle', 'ProCod', 'NumSeq'], keep=False)
registros_duplicados = df_notas[duplicatas_mask]

print(f"📊 Registros com duplicatas: {len(registros_duplicados):,}")

if len(registros_duplicados) > 0:
    print("\n🔧 TRATANDO DUPLICATAS...")
    print("   Estratégia: Manter apenas o último registro (versão corrigida)")
    
    # Removendo duplicatas mantendo o último registro
    df_notas = df_notas.drop_duplicates(subset=['Controle', 'ProCod', 'NumSeq'], keep='last')
    
    duplicatas_removidas = len(registros_duplicados) - len(df_notas[df_notas.duplicated(subset=['Controle', 'ProCod', 'NumSeq'])])
    
    print(f"📊 Duplicatas removidas: {len(registros_duplicados) - len(df_notas):,}")
    print("✅ Duplicatas tratadas com sucesso!")
else:
    print("✅ Nenhuma duplicata encontrada!")

print(f"\n📊 Registros finais: {len(df_notas):,}")

🔍 VERIFICANDO DUPLICATAS...
📊 Registros com duplicatas: 648

🔧 TRATANDO DUPLICATAS...
   Estratégia: Manter apenas o último registro (versão corrigida)
📊 Registros com duplicatas: 648

🔧 TRATANDO DUPLICATAS...
   Estratégia: Manter apenas o último registro (versão corrigida)
📊 Duplicatas removidas: -2,531,778
✅ Duplicatas tratadas com sucesso!

📊 Registros finais: 2,532,426
📊 Duplicatas removidas: -2,531,778
✅ Duplicatas tratadas com sucesso!

📊 Registros finais: 2,532,426


In [14]:
# ========================================================================
# 6. VALIDAÇÃO FINAL DA QUALIDADE DOS DADOS
# ========================================================================

print("✅ VALIDAÇÃO FINAL DA QUALIDADE DOS DADOS")
print("="*50)

# Verificações de qualidade
colunas_importantes = ['Data', 'Controle', 'Cliente', 'Bairro', 'ProCod', 'Quantidade', 'Preco', 'TotalItem', 'ProNom']

print("\n🔍 Verificação de valores nulos:")
problemas_encontrados = 0
for col in colunas_importantes:
    nulos = df_notas[col].isnull().sum()
    vazios = df_notas[col].eq('').sum() if df_notas[col].dtype == 'object' else 0
    if nulos > 0 or vazios > 0:
        print(f"   ⚠️ {col}: {nulos} nulos, {vazios} vazios")
        problemas_encontrados += 1
    else:
        print(f"   ✅ {col}: OK")

print("\n🔍 Verificação de valores inconsistentes:")
# Preços e quantidades
precos_negativos = len(df_notas[df_notas['Preco'] <= 0])
qtd_negativas = len(df_notas[df_notas['Quantidade'] <= 0])

if precos_negativos > 0:
    print(f"   ⚠️ Preços <= 0: {precos_negativos}")
    problemas_encontrados += 1
else:
    print(f"   ✅ Preços: Todos positivos")

if qtd_negativas > 0:
    print(f"   ⚠️ Quantidades <= 0: {qtd_negativas}")
    problemas_encontrados += 1
else:
    print(f"   ✅ Quantidades: Todas positivas")

# Verificação final de duplicatas
duplicatas_restantes = df_notas.duplicated(subset=['Controle', 'ProCod', 'NumSeq']).sum()
if duplicatas_restantes > 0:
    print(f"   ⚠️ Duplicatas restantes: {duplicatas_restantes}")
    problemas_encontrados += 1
else:
    print(f"   ✅ Duplicatas: Nenhuma encontrada")

print(f"\n📊 RESUMO DA VALIDAÇÃO:")
print(f"   • Total de registros: {len(df_notas):,}")
print(f"   • Período: {df_notas['Data'].min().strftime('%d/%m/%Y')} a {df_notas['Data'].max().strftime('%d/%m/%Y')}")
print(f"   • Clientes únicos: {df_notas['Cliente'].nunique():,}")
print(f"   • Produtos únicos: {df_notas['ProNom'].nunique():,}")
print(f"   • Bairros únicos: {df_notas['Bairro'].nunique()}")
print(f"   • Faturamento total: R$ {df_notas['TotalItem'].sum():,.2f}")

if problemas_encontrados == 0:
    print(f"\n🎉 DADOS COMPLETAMENTE LIMPOS E VALIDADOS!")
    print(f"   ✅ Qualidade: EXCELENTE")
    print(f"   ✅ Prontos para análise!")
else:
    print(f"\n⚠️ ENCONTRADOS {problemas_encontrados} PROBLEMAS!")
    print(f"   Revisar antes de prosseguir com análises.")

✅ VALIDAÇÃO FINAL DA QUALIDADE DOS DADOS

🔍 Verificação de valores nulos:
   ✅ Data: OK
   ✅ Controle: OK
   ✅ Cliente: OK
   ✅ Bairro: OK
   ✅ ProCod: OK
   ✅ Quantidade: OK
   ✅ Preco: OK
   ✅ TotalItem: OK
   ✅ Bairro: OK
   ✅ ProCod: OK
   ✅ Quantidade: OK
   ✅ Preco: OK
   ✅ TotalItem: OK
   ✅ ProNom: OK

🔍 Verificação de valores inconsistentes:
   ✅ Preços: Todos positivos
   ✅ Quantidades: Todas positivas
   ✅ ProNom: OK

🔍 Verificação de valores inconsistentes:
   ✅ Preços: Todos positivos
   ✅ Quantidades: Todas positivas
   ✅ Duplicatas: Nenhuma encontrada

📊 RESUMO DA VALIDAÇÃO:
   • Total de registros: 2,532,426
   • Período: 02/01/2024 a 31/12/2024
   • Clientes únicos: 2,521
   ✅ Duplicatas: Nenhuma encontrada

📊 RESUMO DA VALIDAÇÃO:
   • Total de registros: 2,532,426
   • Período: 02/01/2024 a 31/12/2024
   • Clientes únicos: 2,521
   • Produtos únicos: 12,061
   • Bairros únicos: 22
   • Faturamento total: R$ 24,447,211.77

🎉 DADOS COMPLETAMENTE LIMPOS E VALIDADOS!
   ✅

In [15]:
# ========================================================================
# 7. CRIAÇÃO DAS VARIÁVEIS PRINCIPAIS (DADOS LIMPOS)
# ========================================================================

print("📊 CRIANDO VARIÁVEIS PRINCIPAIS COM DADOS LIMPOS...")
print("="*55)

# ========================================================================
# VARIÁVEIS PRINCIPAIS PARA ANÁLISES
# ========================================================================

# Clientes únicos
clientes_unicos = df_notas['Cliente'].unique()
print(f"✅ Clientes únicos: {len(clientes_unicos):,}")

# Compras únicas (notas fiscais)
compras_unicas = df_notas['Controle'].unique()
print(f"✅ Compras únicas: {len(compras_unicas):,}")

# Compras únicas por cliente
compras_unicas_por_cliente = df_notas.groupby('Cliente')['Controle'].nunique()
print(f"✅ Compras por cliente: {len(compras_unicas_por_cliente):,} clientes mapeados")

# Compras únicas por cliente (excluindo cliente genérico 99999)
compras_unicas_por_cliente_tirando_99999 = df_notas[df_notas["Cliente"] != 99999].groupby('Cliente')['Controle'].nunique()
print(f"✅ Compras por cliente identificado: {len(compras_unicas_por_cliente_tirando_99999):,} clientes")

# ========================================================================
# VARIÁVEIS COMPLEMENTARES
# ========================================================================

# Produtos únicos
produtos_unicos = df_notas['ProNom'].unique()
print(f"✅ Produtos únicos: {len(produtos_unicos):,}")

# Bairros únicos (padronizados)
bairros_unicos = df_notas['Bairro'].unique()
print(f"✅ Bairros únicos (padronizados): {len(bairros_unicos)}")

# Período de dados
periodo_inicio = df_notas['Data'].min()
periodo_fim = df_notas['Data'].max()
print(f"✅ Período: {periodo_inicio.strftime('%d/%m/%Y')} a {periodo_fim.strftime('%d/%m/%Y')}")

# Faturamento total
faturamento_total = df_notas['TotalItem'].sum()
print(f"✅ Faturamento total: R$ {faturamento_total:,.2f}")

print(f"\n🎯 VARIÁVEIS PRINCIPAIS CRIADAS COM SUCESSO!")
print(f"   Todas baseadas em dados 100% limpos e validados.")
print(f"   Prontas para análises subsequentes!")

# ========================================================================
# LIMPEZA DE DADOS CONCLUÍDA - INÍCIO DAS ANÁLISES
# ========================================================================

📊 CRIANDO VARIÁVEIS PRINCIPAIS COM DADOS LIMPOS...
✅ Clientes únicos: 2,521
✅ Compras únicas: 302,837
✅ Compras por cliente: 2,521 clientes mapeados
✅ Compras por cliente identificado: 2,520 clientes
✅ Compras por cliente: 2,521 clientes mapeados
✅ Compras por cliente identificado: 2,520 clientes
✅ Produtos únicos: 12,061
✅ Bairros únicos (padronizados): 22
✅ Período: 02/01/2024 a 31/12/2024
✅ Faturamento total: R$ 24,447,211.77

🎯 VARIÁVEIS PRINCIPAIS CRIADAS COM SUCESSO!
   Todas baseadas em dados 100% limpos e validados.
   Prontas para análises subsequentes!
✅ Produtos únicos: 12,061
✅ Bairros únicos (padronizados): 22
✅ Período: 02/01/2024 a 31/12/2024
✅ Faturamento total: R$ 24,447,211.77

🎯 VARIÁVEIS PRINCIPAIS CRIADAS COM SUCESSO!
   Todas baseadas em dados 100% limpos e validados.
   Prontas para análises subsequentes!


# ✅ CONFIRMAÇÃO: TODAS AS ANÁLISES USAM DADOS 100% LIMPOS

**IMPORTANTE:** A partir desta célula, **TODAS** as análises utilizam exclusivamente:

- **`df_notas`** → DataFrame com dados 100% limpos e validados
- **Variáveis criadas na célula anterior** → Baseadas em dados limpos
- **Zero inconsistências** → Padronização completa aplicada

---

### 🎯 **Dados Utilizados nas Análises:**
- ✅ **2.532.426 registros** limpos e validados
- ✅ **Serviços e fretes removidos** (3.570 registros excluídos)  
- ✅ **Bairros padronizados** (28 → 22 bairros únicos)
- ✅ **Duplicatas tratadas** (324 duplicatas removidas)
- ✅ **Qualidade confirmada** (sem nulos, sem inconsistências)

### 📋 **Estrutura das Análises:**
1. **Células 1-7:** Pipeline de limpeza + Criação de variáveis
2. **Células 8+:** Análises baseadas em dados 100% confiáveis

---

# 2. Itens por venda: número médio de itens por venda.

In [16]:
soma = df_notas["Controle"].value_counts().sum() # Somatório de itens DIFERENTES em cada venda
media = soma/len(compras_unicas) # Dividido pela quantidade total de vendas

# Possibildiade: deveríamos considerar a quantidade interna de um item nesse somatório? (Ex: 6 unidades de waffers em uma compra devem contar como 6 ou 1? Atual: 1)
print(f"Média de itens por venda: {media} itens/venda")

# Dá pra fazer histograma com esses dados



Média de itens por venda: 8.362340136773248 itens/venda


# 3. Fidelização de clientes: número de vezes que o cliente retornou para fazer novas compras.

In [17]:
clientes_que_retornaram = compras_unicas_por_cliente[compras_unicas_por_cliente > 1] # De maneira bem simples, podemos simplesmente ver quantos vieram mais de uma vez

# Sugestões:
# 1. Ver clientes que retornaram mais vezes para ver como a fidelização realmente se aplica
clientes_que_retornaram_multiplas_vezes = compras_unicas_por_cliente[compras_unicas_por_cliente > 5]

# 2. Talvez: excluir clientes cuja primeira compra foi muito recente (pode ser um cliente novo)

# Dados
print(f"Clientes: {len(clientes_unicos)} | Retornaram: {len(clientes_que_retornaram)} | Retornaram múltiplas vezes: {len(clientes_que_retornaram_multiplas_vezes)}")
print(f"          100% | Retorna.: {round(len(clientes_que_retornaram)/len(clientes_unicos)*100, 2)}% | Retornaram múltiplas vezes: {round(len(clientes_que_retornaram_multiplas_vezes)/len(clientes_unicos)*100, 2)}%")

# Médias
soma = compras_unicas_por_cliente.sum()
soma2 = compras_unicas_por_cliente_tirando_99999.sum()
print(f"\nMédia de compras por cliente: {round(soma/len(clientes_unicos), 2)} compras/cliente")
print(f"Média de compras por cliente (excluindo o 99999): {round(soma2/len(clientes_unicos), 2)}")

# Dá pra fazer histograma com esses dados (tanto os percentuais de retorno quanto a quantidade de compras)

Clientes: 2521 | Retornaram: 2072 | Retornaram múltiplas vezes: 1466
          100% | Retorna.: 82.19% | Retornaram múltiplas vezes: 58.15%

Média de compras por cliente: 124.36 compras/cliente
Média de compras por cliente (excluindo o 99999): 30.46


# 4. Número de clientes: quantidade de pessoas que compraram no estabelecimento.

In [18]:
print(f"Clientes únicos: {len(clientes_unicos)} (-1 se desconsiderar o 99999)") # A contagem foi feita pelo tamanho da lista de clientes únicos (que possuem Cód. de Cliente distintos)

Clientes únicos: 2521 (-1 se desconsiderar o 99999)


# 5. Vendas por bairro: quantidade de vendas por bairros/locais de entrega.

In [ ]:
# ========================================================================
# 5. VENDAS POR BAIRRO (DADOS LIMPOS)
# ========================================================================

# Utilizando dados 100% limpos (df_notas)
df_por_local = df_notas.groupby(["Bairro"]).size().sort_values(ascending=False)

print("📊 VENDAS POR BAIRRO (dados limpos e padronizados):")
print("="*55)

for bairro, qtd_vendas in df_por_local.head(10).items():
    percentual = (qtd_vendas / df_por_local.sum()) * 100
    print(f"   {bairro:<25} {qtd_vendas:>6,} vendas ({percentual:>5.1f}%)")

print(f"\n✅ Total de bairros únicos: {len(df_por_local)}")
print(f"✅ Bairros foram padronizados na limpeza de dados")

df_por_local

# 6. Clientes com maiores vendas: identificação do perfil do cliente (valor de vendas, ticket médio, frequência de retorno, itens adquiridos, ...).

In [ ]:
# ========================================================================
# 6. PERFIL DE CLIENTES IDENTIFICADOS (DADOS LIMPOS)
# ========================================================================

# Data mais recente no dataset para cálculo de recência
# Utilizando dados 100% limpos (df_notas)
data_analise = df_notas['Data'].max()

print("--- ANÁLISE DE PERFIL: CLIENTES IDENTIFICADOS (EXCLUINDO 99999) ---")
print("    Baseada em dados 100% limpos e validados")
print("="*70)

df_clientes_identificados = df_notas[df_notas['Cliente'] != 99999].copy()

# 1. Selecionar os clientes com maior quantidade de vendas (Top 10)
# A 'quantidade de vendas' pode ser interpretada como o valor total vendido ou o número de compras (controles únicos).
# Vamos considerar o valor total vendido.

valor_vendas_por_cliente_identificado = df_clientes_identificados.groupby('Cliente')['TotalItem'].sum()
clientes_maior_valor_vendas = valor_vendas_por_cliente_identificado.nlargest(10)

print("\n1. Clientes com Maior Valor Total de Vendas:")
print(clientes_maior_valor_vendas)

# 2. Definir o ticket médio por cliente
# Contagem de compras únicas por cliente identificado
compras_unicas_por_cliente_identificado = df_clientes_identificados.groupby('Cliente')['Controle'].nunique()
# Ticket médio = Valor Total de Vendas / Número de Compras Únicas
ticket_medio_clientes_identificados = (valor_vendas_por_cliente_identificado / compras_unicas_por_cliente_identificado).dropna()

print("\n2. Ticket Médio por Cliente (Top 10):")
print(ticket_medio_clientes_identificados.nlargest(10))

# 3. Frequência de retorno (Número de compras únicas por cliente)
# Já calculado acima: compras_unicas_por_cliente_identificado
print("\n3. Frequência de Retorno (Número de Compras Únicas por Cliente - Top 10):")
print(compras_unicas_por_cliente_identificado.nlargest(10))

# 4. Principais itens adquiridos por esses clientes (Top 10 produtos em valor ou quantidade)
# Para os clientes identificados como um todo:

itens_adquiridos_clientes_identificados = df_clientes_identificados.groupby('ProNom')['TotalItem'].sum().nlargest(10)

print("\n4. Principais Itens Adquiridos pelos Clientes Identificados (Top 10 por Valor Total Vendido):")
print(itens_adquiridos_clientes_identificados)

# **Bônus: Perfil RFV (Recência, Frequência, Valor) para clientes identificados**
# Recência: Dias desde a última compra

# Juntar todas as métricas em um DataFrame de perfil
df_perfil_clientes_identificados = pd.DataFrame({
    'Total_Vendas': valor_vendas_por_cliente_identificado,
    'Num_Compras_Unicas': compras_unicas_por_cliente_identificado,
    'Ticket_Medio': ticket_medio_clientes_identificados,
}).sort_values(by='Total_Vendas', ascending=False) # Ordenar pelos maiores valores de vendas

print("\n--- DataFrame de Perfil dos Clientes Identificados (Top 10) ---")
print(df_perfil_clientes_identificados.head(10))

In [ ]:
# ========================================================================
# 6.2 PERFIL DO CLIENTE GENÉRICO 99999 (DADOS LIMPOS)
# ========================================================================

print("--- ANÁLISE DE PERFIL: CLIENTE GENÉRICO (99999) ---")
print("    Baseada em dados 100% limpos e validados")
print("="*55)

# Utilizando dados 100% limpos (df_notas)
df_cliente_99999 = df_notas[df_notas['Cliente'] == 99999].copy()

if not df_cliente_99999.empty:
    # 1. Valor de vendas para o cliente 99999
    valor_vendas_99999 = df_cliente_99999['TotalItem'].sum()
    print(f"\n1. Valor Total de Vendas do Cliente 99999: R$ {valor_vendas_99999:.2f}")

    # 2. Definir o ticket médio para o cliente 99999
    num_compras_unicas_99999 = df_cliente_99999['Controle'].nunique()
    ticket_medio_99999 = valor_vendas_99999 / num_compras_unicas_99999 if num_compras_unicas_99999 > 0 else 0

    print(f"\n2. Ticket Médio do Cliente 99999: R$ {ticket_medio_99999:.2f}")

    # 3. Frequência de retorno (Número de compras únicas) para o cliente 99999
    print(f"\n3. Frequência de Retorno (Número de Compras Únicas) do Cliente 99999: {num_compras_unicas_99999}")

    # 4. Principais itens adquiridos pelo cliente 99999 (Top 10 produtos em valor ou quantidade)
    itens_adquiridos_99999 = df_cliente_99999.groupby('ProNom')['TotalItem'].sum().nlargest(10)

    print("\n4. Principais Itens Adquiridos pelo Cliente 99999 (Top 10 por Valor Total Vendido):")
    print(itens_adquiridos_99999)

    # **Bônus: Perfil RFV para o cliente 99999**
    # Agora trabalhando com datetime corretamente
    data_analise = df_notas['Data'].max()
    ultima_compra_99999 = df_cliente_99999['Data'].max()
    recencia_99999 = (data_analise - ultima_compra_99999).days

    print(f"\nRecência do Cliente 99999 (dias desde a última compra): {recencia_99999} dias")

    print("\n--- Resumo do Perfil do Cliente 99999 ---")
    print(f"Total de Vendas: R$ {valor_vendas_99999:.2f}")
    print(f"Número de Compras Únicas: {num_compras_unicas_99999}")
    print(f"Ticket Médio: R$ {ticket_medio_99999:.2f}")
    print(f"Recência (Dias): {recencia_99999}")

else:
    print("O cliente 99999 não foi encontrado no DataFrame.")

# 7. Volume de vendas: Total de faturamento de um período.

In [ ]:
# ========================================================================
# 7. VOLUME DE VENDAS - ANÁLISE TEMPORAL (DADOS LIMPOS)
# ========================================================================

print("=== VOLUME DE VENDAS - ANÁLISE TEMPORAL ===")
print("    Baseada em dados 100% limpos e validados")
print("="*55)

# Volume total do período - utilizando dados 100% limpos (df_notas)
volume_total = df_notas['TotalItem'].sum()
periodo_inicio = df_notas['Data'].min()
periodo_fim = df_notas['Data'].max()

print(f"Volume total de faturamento: R$ {volume_total:,.2f}")
print(f"Período analisado: {periodo_inicio.strftime('%d/%m/%Y')} a {periodo_fim.strftime('%d/%m/%Y')}")

# Volume por mês
volume_por_mes = df_notas.groupby('Mes')['TotalItem'].sum().sort_index()
print(f"\n--- Volume por Mês ---")
for mes, volume in volume_por_mes.items():
    print(f"Mês {mes:2d}: R$ {volume:,.2f}")

# Volume por semana (primeiras 10 semanas como exemplo)
volume_por_semana = df_notas.groupby('Semana')['TotalItem'].sum().sort_index()
print(f"\n--- Volume por Semana (Top 10 semanas) ---")
top_semanas = volume_por_semana.nlargest(10)
for semana, volume in top_semanas.items():
    print(f"Semana {semana:2d}: R$ {volume:,.2f}")

# Estatísticas básicas
print(f"\n--- Estatísticas do Volume de Vendas ---")
print(f"Média mensal: R$ {volume_por_mes.mean():,.2f}")
print(f"Maior volume mensal: R$ {volume_por_mes.max():,.2f} (Mês {volume_por_mes.idxmax()})")
print(f"Menor volume mensal: R$ {volume_por_mes.min():,.2f} (Mês {volume_por_mes.idxmin()})")

# Percentual de vendas com CPF vs sem CPF
vendas_com_cpf = df_notas[df_notas['Cliente'] != 99999]['TotalItem'].sum()
vendas_sem_cpf = df_notas[df_notas['Cliente'] == 99999]['TotalItem'].sum()
percentual_com_cpf = (vendas_com_cpf / volume_total) * 100
percentual_sem_cpf = (vendas_sem_cpf / volume_total) * 100

print(f"\n--- Análise CPF vs Vendas ---")
print(f"Vendas com CPF informado: R$ {vendas_com_cpf:,.2f} ({percentual_com_cpf:.1f}%)")
print(f"Vendas sem CPF (99999): R$ {vendas_sem_cpf:,.2f} ({percentual_sem_cpf:.1f}%)")

In [ ]:
# ========================================================================
# 8. ANÁLISES ESPECÍFICAS COTRIFRED (DADOS LIMPOS)
# ========================================================================

print("=== ANÁLISES ESPECÍFICAS COTRIFRED ===")
print("    Baseada em dados 100% limpos e validados")
print("="*55)

# 1. Concentração de clientes por bairro (já foi feita anteriormente, vamos aprimorar)
print("\n1. CONCENTRAÇÃO DE CLIENTES POR BAIRRO")

# Clientes únicos por bairro (excluindo 99999) - utilizando dados limpos
clientes_por_bairro = df_notas[df_notas['Cliente'] != 99999].groupby('Bairro')['Cliente'].nunique().sort_values(ascending=False)
vendas_por_bairro = df_notas.groupby('Bairro')['TotalItem'].sum().sort_values(ascending=False)

print("Top 10 bairros por número de clientes únicos:")
print(clientes_por_bairro.head(10))

print("\nTop 10 bairros por volume de vendas:")
for bairro, valor in vendas_por_bairro.head(10).items():
    print(f"{bairro}: R$ {valor:,.2f}")

# 2. Curva ABC das vendas dos produtos
print("\n\n2. CURVA ABC DOS PRODUTOS")

# Agrupando vendas por produto - utilizando dados limpos
vendas_por_produto = df_notas.groupby('ProNom')['TotalItem'].sum().sort_values(ascending=False)
vendas_por_produto_df = vendas_por_produto.reset_index()
vendas_por_produto_df['Percentual_Individual'] = (vendas_por_produto_df['TotalItem'] / vendas_por_produto_df['TotalItem'].sum()) * 100
vendas_por_produto_df['Percentual_Acumulado'] = vendas_por_produto_df['Percentual_Individual'].cumsum()

# Classificação ABC
def classificar_abc(percentual_acum):
    if percentual_acum <= 80:
        return 'A'
    elif percentual_acum <= 95:
        return 'B'
    else:
        return 'C'

vendas_por_produto_df['Classe_ABC'] = vendas_por_produto_df['Percentual_Acumulado'].apply(classificar_abc)

# Contagem por classe
abc_summary = vendas_por_produto_df['Classe_ABC'].value_counts()
print("Distribuição de produtos por classe ABC:")
print(f"Classe A: {abc_summary.get('A', 0)} produtos (80% do faturamento)")
print(f"Classe B: {abc_summary.get('B', 0)} produtos (15% do faturamento)")  
print(f"Classe C: {abc_summary.get('C', 0)} produtos (5% do faturamento)")

print("\nTop 20 produtos da Classe A:")
produtos_classe_a = vendas_por_produto_df[vendas_por_produto_df['Classe_ABC'] == 'A'].head(20)
for _, row in produtos_classe_a.iterrows():
    print(f"{row['ProNom'][:50]:<50} R$ {row['TotalItem']:>10,.2f} ({row['Percentual_Individual']:.1f}%)")

# 3. Identificar itens "não movimentados" da família 1 (MERCADO)
print("\n\n3. ANÁLISE DE PRODUTOS DA FAMÍLIA MERCADO")

# Verificando grupos disponíveis - utilizando dados limpos
grupos_disponiveis = df_notas['GrpNom'].unique()
print("Grupos disponíveis no dataset:")
for grupo in sorted(grupos_disponiveis):
    count = df_notas[df_notas['GrpNom'] == grupo]['ProNom'].nunique()
    print(f"- {grupo}: {count} produtos únicos")

# Identificando produtos do MERCADO (assumindo que são produtos alimentícios)
# Como não temos uma família específica "MERCADO", vamos filtrar os principais grupos alimentícios
grupos_mercado = [
    'FRUTAS,VERDUR,OVOS,CARV,FLO NA',
    'PADARIA E CONFEITARIA', 
    'BEBIDAS',
    'AÇOUGUE',
    'MERCEARIA',
    'LATICÍNIOS'
]

produtos_mercado = df_notas[df_notas['GrpNom'].isin(grupos_mercado)]

print(f"\nProdutos da categoria MERCADO analisados: {produtos_mercado['ProNom'].nunique()} produtos únicos")
print(f"Total de vendas da categoria MERCADO: R$ {produtos_mercado['TotalItem'].sum():,.2f}")

# Produtos com baixa movimentação (menos de 10 vendas no período)
movimentacao_produtos_mercado = produtos_mercado.groupby('ProNom').agg({
    'Controle': 'nunique',  # Número de vendas diferentes
    'TotalItem': 'sum',     # Valor total vendido
    'Quantidade': 'sum'     # Quantidade total vendida
}).sort_values('Controle')

produtos_baixa_mov = movimentacao_produtos_mercado[movimentacao_produtos_mercado['Controle'] <= 10]
print(f"\nProdutos com baixa movimentação (≤10 vendas): {len(produtos_baixa_mov)} produtos")

print("\nTop 20 produtos com menor movimentação:")
for produto, dados in produtos_baixa_mov.head(20).iterrows():
    print(f"{produto[:50]:<50} {dados['Controle']:>3} vendas | R$ {dados['TotalItem']:>8,.2f}")

print(f"\nPRODUTOS COM APENAS 1 VENDA: {len(movimentacao_produtos_mercado[movimentacao_produtos_mercado['Controle'] == 1])} produtos")

In [24]:
# RESUMO EXECUTIVO - PRINCIPAIS INSIGHTS

print("="*80)
print("                        RESUMO EXECUTIVO")
print("                   ANÁLISE DE DADOS - COTRIFRED")
print("="*80)

# Dados gerais
total_clientes = len(df_notas['Cliente'].unique())
total_vendas = len(df_notas['Controle'].unique())
faturamento_total = df_notas['TotalItem'].sum()

print(f"\n📊 DADOS GERAIS:")
print(f"   • Total de clientes: {total_clientes:,}")
print(f"   • Total de vendas: {total_vendas:,}")
print(f"   • Faturamento total: R$ {faturamento_total:,.2f}")
print(f"   • Período: {df_notas['Data'].min().strftime('%d/%m/%Y')} a {df_notas['Data'].max().strftime('%d/%m/%Y')}")

# CPF vs Sem CPF
clientes_sem_cpf = len(df_notas[df_notas['Cliente'] == 99999]['Controle'].unique())
clientes_com_cpf = total_vendas - clientes_sem_cpf
vendas_sem_cpf = df_notas[df_notas['Cliente'] == 99999]['TotalItem'].sum()
vendas_com_cpf = faturamento_total - vendas_sem_cpf

print(f"\n🆔 ANÁLISE CPF:")
print(f"   • Vendas SEM CPF: {clientes_sem_cpf:,} ({clientes_sem_cpf/total_vendas*100:.1f}%) = R$ {vendas_sem_cpf:,.2f}")
print(f"   • Vendas COM CPF: {clientes_com_cpf:,} ({clientes_com_cpf/total_vendas*100:.1f}%) = R$ {vendas_com_cpf:,.2f}")

# Fidelização
clientes_identificados = df_notas[df_notas['Cliente'] != 99999]['Cliente'].unique()
retornaram = len(df_notas[df_notas['Cliente'] != 99999].groupby('Cliente')['Controle'].nunique()[df_notas[df_notas['Cliente'] != 99999].groupby('Cliente')['Controle'].nunique() > 1])

print(f"\n🔄 FIDELIZAÇÃO:")
print(f"   • Clientes identificados: {len(clientes_identificados)}")
print(f"   • Clientes que retornaram: {retornaram} ({retornaram/len(clientes_identificados)*100:.1f}%)")

# Itens por venda
itens_por_venda = df_notas['Controle'].value_counts().sum() / len(df_notas['Controle'].unique())
print(f"\n🛒 COMPORTAMENTO DE COMPRA:")
print(f"   • Média de itens por venda: {itens_por_venda:.1f} itens")

# Top bairros
top_bairros = df_notas.groupby('Bairro')['TotalItem'].sum().nlargest(3)
print(f"\n🏘️  TOP 3 BAIRROS (por faturamento):")
for i, (bairro, valor) in enumerate(top_bairros.items(), 1):
    print(f"   {i}. {bairro}: R$ {valor:,.2f}")

# Sazonalidade
volume_por_mes = df_notas.groupby('Mes')['TotalItem'].sum()
mes_maior = volume_por_mes.idxmax()
mes_menor = volume_por_mes.idxmin()

print(f"\n📅 SAZONALIDADE:")
print(f"   • Maior volume: Mês {mes_maior} (R$ {volume_por_mes[mes_maior]:,.2f})")
print(f"   • Menor volume: Mês {mes_menor} (R$ {volume_por_mes[mes_menor]:,.2f})")

print(f"\n💡 PRINCIPAIS OPORTUNIDADES:")
print(f"   • 62% das vendas são sem CPF - grande potencial para programas de fidelidade")
print(f"   • 82% dos clientes identificados retornam - boa base para marketing direto")
print(f"   • Centro concentra maior volume - otimizar logística para esta região")
print(f"   • Dezembro é o mês de maior faturamento - preparar estoque antecipadamente")

print("="*80)

                        RESUMO EXECUTIVO
                   ANÁLISE DE DADOS - COTRIFRED

📊 DADOS GERAIS:
   • Total de clientes: 2,521
   • Total de vendas: 302,837
   • Faturamento total: R$ 24,447,211.77
   • Período: 02/01/2024 a 31/12/2024

🆔 ANÁLISE CPF:
   • Vendas SEM CPF: 236,723 (78.2%) = R$ 15,195,893.42
   • Vendas COM CPF: 66,114 (21.8%) = R$ 9,251,318.34

🆔 ANÁLISE CPF:
   • Vendas SEM CPF: 236,723 (78.2%) = R$ 15,195,893.42
   • Vendas COM CPF: 66,114 (21.8%) = R$ 9,251,318.34

🔄 FIDELIZAÇÃO:
   • Clientes identificados: 2520
   • Clientes que retornaram: 2071 (82.2%)

🛒 COMPORTAMENTO DE COMPRA:
   • Média de itens por venda: 8.4 itens

🏘️  TOP 3 BAIRROS (por faturamento):
   1. CENTRO: R$ 17,274,654.78
   2. INTERIOR: R$ 3,631,593.27
   3. APARECIDA: R$ 853,507.06

🔄 FIDELIZAÇÃO:
   • Clientes identificados: 2520
   • Clientes que retornaram: 2071 (82.2%)

🛒 COMPORTAMENTO DE COMPRA:
   • Média de itens por venda: 8.4 itens

🏘️  TOP 3 BAIRROS (por faturamento):
   1. CEN